In [2]:
import plotly.graph_objects as go
import pandas as pd

In [3]:
df_medals = pd.read_excel("data/Medals.xlsx")
print(df_medals.info())
df_medals.rename(columns={'Team/NOC':'Country', 'Total': 'Total Medals', 'Gold':'Gold Medals', 'Silver': 'Silver Medals', 'Bronze': 'Bronze Medals'}, inplace=True)
df_medals.drop(columns=['Unnamed: 7','Unnamed: 8','Rank by Total'], inplace=True)
df_medals

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 93 entries, 0 to 92
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Rank           93 non-null     int64  
 1   Team/NOC       93 non-null     object 
 2   Gold           93 non-null     int64  
 3   Silver         93 non-null     int64  
 4   Bronze         93 non-null     int64  
 5   Total          93 non-null     int64  
 6   Rank by Total  93 non-null     int64  
 7   Unnamed: 7     0 non-null      float64
 8   Unnamed: 8     1 non-null      float64
dtypes: float64(2), int64(6), object(1)
memory usage: 6.7+ KB
None


,Rank,Country,Gold Medals,Silver Medals,Bronze Medals,Total Medals
0,1,United States of America,39,41,33,113
1,2,People's Republic of China,38,32,18,88
2,3,Japan,27,14,17,58
3,4,Great Britain,22,21,22,65
4,5,ROC,20,28,23,71
...,...,...,...,...,...,...
88,86,Ghana,0,0,1,1
89,86,Grenada,0,0,1,1
90,86,Kuwait,0,0,1,1
91,86,Republic of Moldova,0,0,1,1


In [18]:
def node_label_to_color(label):
    if "New" in label:
        return "palevioletred"
    if "Pending" in label:
        return "silver"
    if "Triage Backlog" in label:
        return "orange"
    if "Bug Backlog" in label:
        return "dodgerblue"
    if "Closed" in label:
        return "seagreen"
    return "black"

In [ ]:
pending_last_week = 15
pending_this_week = 20

In [15]:
start_date = "11/13"
end_date = "11/17"
basic_labels = ["Pending", "Triage Backlog", "Bug Backlog"]


last_week_labels = ["New Tickets from T2"] + [f"{label} Last Week" for label in basic_labels]
this_week_start_labels = [f"{label} {start_date}" for label in basic_labels]
this_week_middle_labels = []
this_week_end_labels = [f"{label} {end_date}" for label in basic_labels] + ["Closed from Triage", "Closed From Bug"]

# Initialize nodes
node_labels = last_week_labels + this_week_start_labels + this_week_end_labels
node_colors = [node_label_to_color(label) for label in node_labels]
# actually the x coordinates
node_y_pos = []
# actually the y coordinates
node_x_pos = []

# Last week node positions
for i in range(len(last_week_labels)):
    node_y_pos.append(0.01)
    node_x_pos.append(0.01 + i * 0.3)

# This week start node positions
for i in range(len(this_week_start_labels)):
    node_y_pos.append(0.5)
    node_x_pos.append(0.01 + i * 0.3)
    
# This week end node positions
for i in range(len(this_week_end_labels)):
    node_y_pos.append(0.75)
    node_x_pos.append(0.01 + i * 0.3)

# Initialize links
link_labels = []
# Get from Jira
link_values = []
link_source = []
link_target = []
link_colors = []

# From last week to this week
last_to_this_week_source = [0, 1, 2, 3]
last_to_this_week_target = [0, 0, 1, 2]
link_source += last_to_this_week_source
link_target += last_to_this_week_target


for source, target in zip(link_source, link_target):
    link_labels.append(f"{last_week_labels[source]} to {this_week_start_labels[target]}")

# From this week start to this week end
# Pending, Triage Backlog, Bug Backlog
# Pending, Triage Backlog, Bug Backlog, Closed from Triage, Closed from Bug
this_week_start_to_end_source = [0, 0, 1, 1, ]
this_week_start_to_end_target = [0, 1, 1, 2, ]
link_source += this_week_start_to_end_source
link_target += this_week_start_to_end_target

for source, target in zip(this_week_start_to_end_source, this_week_start_to_end_target):
    link_labels.append(f"{this_week_start_labels[source]} to {this_week_end_labels[target]}")


link_values = [22]
link_colors = ["pink"]



In [16]:
last_week_labels

['New Tickets from T2',
 'Pending Last Week',
 'Triage Backlog Last Week',
 'Bug Backlog Last Week']

In [11]:
# Display the figure
NODES = dict(pad  = 20, thickness = 20, line  = dict(color = "lightslategrey", width = 0.5), hovertemplate=" ",
            label = node_labels, color = node_colors,
            x = node_x_pos, y = node_y_pos, )
LINKS = dict(source = link_source, target = link_target, value = link_values, 
            label = link_labels, color = link_colors,
            hovertemplate="%{label}",)
data = go.Sankey( arrangement='snap', node = NODES, link = LINKS)
fig = go.Figure(data)
fig.update_traces( valueformat='3d', valuesuffix=' Tickets', selector=dict(type='sankey'))
fig.update_layout(title="Triage Stats 11/13/23 to 11/17/23",  font_size=16,  width=1200,height=500,)
fig.update_layout(hoverlabel=dict( bgcolor="grey", font_size=14, font_family="Rockwell"))
fig.show()

In [4]:
NUM_COUNTRIES = 5
X_POS, Y_POS = 0.5, 1/(NUM_COUNTRIES-1)
NODE_COLORS = ["seagreen", "dodgerblue", "orange", "palevioletred", "darkcyan"]
LINK_COLORS = ["lightgreen", "lightskyblue", "bisque", "pink", "lightcyan"]

source = []
node_x_pos, node_y_pos = [], []
node_labels, node_colors = [], NODE_COLORS[0:NUM_COUNTRIES]
link_labels, link_colors, link_values = [], [], [] 

# FIRST set of links and nodes
for i in range(NUM_COUNTRIES):
    source.extend([i]*3)
    node_x_pos.append(0.01)
    node_y_pos.append(round(i*Y_POS+0.01,2))
    country = df_medals['Country'][i]
    node_labels.append(country) 
    for medal in ["Gold", "Silver", "Bronze"]:
        link_labels.append(f"{country}-{medal}")
        link_values.append(df_medals[f"{medal} Medals"][i])
    link_colors.extend([LINK_COLORS[i]]*3)

source_last = max(source)+1
target = [ source_last, source_last+1, source_last+2] * NUM_COUNTRIES
target_last = max(target)+1

node_labels.extend(["Gold", "Silver", "Bronze"])
node_colors.extend(["gold", "silver", "brown"])
node_x_pos.extend([X_POS, X_POS, X_POS])
node_y_pos.extend([0.01, 0.5, 1])

# LAST set of links and nodes
source.extend([ source_last, source_last+1, source_last+2])
target.extend([target_last]*3)
node_labels.extend(["Total Medals"])
node_colors.extend(["grey"])
node_x_pos.extend([X_POS+0.25])
node_y_pos.extend([0.5])

for medal in ["Gold","Silver","Bronze"]:
    link_labels.append(f"{medal}")
    link_values.append(df_medals[f"{medal} Medals"][:i+1].sum())
link_colors.extend(["gold", "silver", "brown"])

print("source", source, source_last)
print("target", target)
print("node_labels", node_labels)
print("node_x_pos", node_x_pos)
print("node_y_pos", node_y_pos)
print("node_colors", node_colors)
print("link_labels", link_labels)
print("link_values", link_values)
print("link_colors", link_colors)

source [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 6, 7] 5
target [5, 6, 7, 5, 6, 7, 5, 6, 7, 5, 6, 7, 5, 6, 7, 8, 8, 8]
node_labels ['United States of America', "People's Republic of China", 'Japan', 'Great Britain', 'ROC', 'Gold', 'Silver', 'Bronze', 'Total Medals']
node_x_pos [0.01, 0.01, 0.01, 0.01, 0.01, 0.5, 0.5, 0.5, 0.75]
node_y_pos [0.01, 0.26, 0.51, 0.76, 1.01, 0.01, 0.5, 1, 0.5]
node_colors ['seagreen', 'dodgerblue', 'orange', 'palevioletred', 'darkcyan', 'gold', 'silver', 'brown', 'grey']
link_labels ['United States of America-Gold', 'United States of America-Silver', 'United States of America-Bronze', "People's Republic of China-Gold", "People's Republic of China-Silver", "People's Republic of China-Bronze", 'Japan-Gold', 'Japan-Silver', 'Japan-Bronze', 'Great Britain-Gold', 'Great Britain-Silver', 'Great Britain-Bronze', 'ROC-Gold', 'ROC-Silver', 'ROC-Bronze', 'Gold', 'Silver', 'Bronze']
link_values [39, 41, 33, 38, 32, 18, 27, 14, 17, 22, 21, 22, 20, 28, 23, 146, 13

In [10]:

# Display the figure
NODES = dict(pad  = 20, thickness = 20, line  = dict(color = "lightslategrey", width = 0.5),hovertemplate=" ",
            label = node_labels, color = node_colors,
            x = node_x_pos, y = node_y_pos, )
LINKS = dict(source = source, target = target, value = link_values, 
            label = link_labels, color = link_colors,
            hovertemplate="%{label}",)
data = go.Sankey( arrangement='snap', node = NODES, link = LINKS)
fig = go.Figure(data)
fig.update_traces( valueformat='3d', valuesuffix=' Medals', selector=dict(type='sankey'))
fig.update_layout(title="Olympics - 2021: Country &  Medals",  font_size=16,  width=1200,height=500,)
fig.update_layout(hoverlabel=dict( bgcolor="grey", font_size=14, font_family="Rockwell"))
fig.show()